In [46]:
from datasets import load_from_disk
import datasets
from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import AutoModelForSequenceClassification
from torch.optim import AdamW
from transformers import get_scheduler
import torch
from tqdm.auto import tqdm
import evaluate
import random
import argparse
from nltk.corpus import wordnet as wn
from nltk import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk import pos_tag

In [4]:
def custom_transform(example):
    
    ################################
    ##### YOUR CODE BEGINGS HERE ###
    
    # Design and implement the transformation as mentioned in pdf
    # You are free to implement any transformation but the comments at the top roughly describe
    # how you could implement two of them --- synonym replacement and typos.
    
    # You should update example["text"] using your transformation
        
    raise NotImplementedError
    
    
    ##### YOUR CODE ENDS HERE ######
    
    return example




In [5]:
# Define the directory where the dataset is saved
load_directory = "dataset/"

# Load the dataset from the specified directory
loaded_data = load_from_disk(load_directory)

# Now you can access the individual splits (train, test, unsupervised) as follows:
train_dataset = loaded_data["train"]
test_dataset = loaded_data["test"]
unsupervised_dataset = loaded_data["unsupervised"]

In [22]:
# replacement probability
probability = 0.6

# Generate a random number between 0 and 1
random_number = random.random()

In [18]:
example = test_dataset[0]['text']

In [34]:
word_list  = word_tokenize(example)

In [49]:
chosen_word = random.choice(word_list)
synonyms = wn.synsets(chosen_word)

# If chosen word doesn't have a synonym select retry until finding a word that does.
max_retry = len(word_list)
retry = 0 
while (len(synonyms) == 0 and retry <= max_retry):
    chosen_word = random.choice(word_list)
    synonyms = wn.synsets(chosen_word)

In [55]:
# Perform POS tagging
pos_tags = pos_tag(word_list)

# Print the word and its POS tag
chosen_word_tag = {tag for word, tag in pos_tags if word == chosen_word}

In [58]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wn.ADJ
    elif treebank_tag.startswith('V'):
        return wn.VERB
    elif treebank_tag.startswith('N'):
        return wn.NOUN
    elif treebank_tag.startswith('R'):
        return wn.ADV
    else:
        return None  # If no match is found, return None

In [45]:
synonym_list = set()
for syn in synonyms: # ADJ, ADJ_SAT, ADV, NOUN or VERB
    for lemma in syn.lemmas():
        if lemma.name() != chosen_word:
            synonym_list.add(lemma.name().replace('_', ' ')) # Replace underscores for multi-word synonyms

In [31]:
synonym_list

{'accept',
 'acquire',
 'admit',
 'adopt',
 'aim',
 'ask',
 'assume',
 'bring',
 'call for',
 'carry',
 'charter',
 'choose',
 'claim',
 'conduct',
 'consider',
 'consume',
 'contain',
 'contract',
 'convey',
 'deal',
 'demand',
 'direct',
 'drive',
 'engage',
 'exact',
 'fill',
 'film',
 'get',
 'get hold of',
 'guide',
 'have',
 'hire',
 'hold',
 'ingest',
 'involve',
 'issue',
 'lead',
 'learn',
 'lease',
 'look at',
 'make',
 'necessitate',
 'need',
 'occupy',
 'pack',
 'payoff',
 'pick out',
 'postulate',
 'proceeds',
 'read',
 'remove',
 'rent',
 'require',
 'return',
 'select',
 'shoot',
 'strike',
 'study',
 'submit',
 'subscribe',
 'subscribe to',
 'take aim',
 'take away',
 'take in',
 'take on',
 'take up',
 'takings',
 'train',
 'use up',
 'withdraw',
 'yield'}